In [1]:
from os import listdir
from os.path import isfile, join
import re
import pickle as pk
from imp import reload

from tagging import *

In [2]:
"""
import sys, importlib
importlib.reload(sys.modules['tagging'])
from tagging import *
"""

"\nimport sys, importlib\nimportlib.reload(sys.modules['tagging'])\nfrom tagging import *\n"

In [3]:
dir_train_s = '/home/bwlee/data/NER/exobrain'
dir_val_s = '/home/bwlee/data/NER/exobrain_valid'

files_train = [join(dir_train_s, f_s) for f_s in listdir(dir_train_s) 
             if isfile(join(dir_train_s, f_s))]
files_val = [join(dir_val_s, f_s) for f_s in listdir(dir_val_s) 
             if isfile(join(dir_val_s, f_s))]
files_test = ['test_exo_form2.txt']

In [20]:
def read_a_file(f_s):
    with open(f_s) as f1:
        flag_text = False
        text, tag_text = [], []
        ii = 1
        for line in f1:
            if line[:2] != '##':
                ii += 1
                continue
            
            if re.match('## \d+$', line):
                flag_text = True
                continue
            else:
                if flag_text:
                    text.append(line[3:])
                    flag_text = False
                else:
                    tag_text.append(line[3:])
                    try:
                        assert len(text) == len(tag_text)
                    except:
                        print(f_s, ii)
                        raise Exception
            ii += 1
            
    return text, tag_text

sent_train = []
tag_sent_train = []
for f_s in files_train:
    sent1, tag_sent1 = read_a_file(f_s)
    sent_train.extend(sent1)
    tag_sent_train.extend(tag_sent1)
    
sent_val = []
tag_sent_val = []
for f_s in files_val:
    sent1, tag_sent1 = read_a_file(f_s)
    sent_val.extend(sent1)
    tag_sent_val.extend(tag_sent1)
    
sent_test = []
tag_sent_test = []
for f_s in files_test:
    sent1, tag_sent1 = read_a_file(f_s)
    sent_test.extend(sent1)
    tag_sent_test.extend(tag_sent1)    

In [21]:
tag_sent_test[:2]

['<삼성전자:ORG>가 디스플레이 관련 일회성 수익에 힘입어 시장 예상을 큰 폭 뛰어넘는 실적을 냈다.\n',
 '<삼성전자:ORG>는 올해 <2분기:NOH> 연결 기준 영업이익이 전년 동기 대비 <22.73%:PNT> 증가한 <8조1천억원:MNY>으로 잠정 집계됐다고 <8일:DAT> 공시했다.\n']

In [22]:
for ii in range(1):
    print(sent_test[ii])
    print(tag_sent_test[ii])

삼성전자가 디스플레이 관련 일회성 수익에 힘입어 시장 예상을 큰 폭 뛰어넘는 실적을 냈다.

<삼성전자:ORG>가 디스플레이 관련 일회성 수익에 힘입어 시장 예상을 큰 폭 뛰어넘는 실적을 냈다.



In [23]:
# SKT KoBERT
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# KoBERT tokenizer
kobert, vocab = get_pytorch_kobert_model()
tokenizer = nlp.data.BERTSPTokenizer(get_tokenizer(), vocab, lower=False)

using cached model
using cached model
using cached model


In [24]:
from transformers import BertTokenizer, BertModel, BertConfig

# BERT tokenizer
B_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [25]:
ne_exp = '<(.+?):([A-Z]{3})>'

def get_token_tag(text, tag_text, tokenizer):
    wp_tokens = token_loc(text, tokenizer)
    ner_tokens = get_NE(tag_text, text, ne_exp)

    # get matching ids
    matching_ids = match(ner_tokens, wp_tokens)

    # preprocess (TAG --> B-TAG or I-TAG)
    tg_tags = [(i, x[1]) for i, x in enumerate(ner_tokens)]
    tags = [None]*len(wp_tokens)
    for ii, tag in tg_tags:
        i_id, f_id = matching_ids[ii]
        tags[i_id] = 'B-'+tag
        for jj in range(i_id+1, f_id+1):
            tags[jj] = 'I-'+tag
    return [ x[0] for x in wp_tokens], [x if x is not None else 'O' for x in tags]

In [26]:
sentences_train, labels_train = [], []
for sent1, tag_sent1 in zip(sent_train, tag_sent_train):
    tokens, tags = get_token_tag(sent1, tag_sent1, tokenizer)
    sentences_train.append(tokens)
    labels_train.append(tags)
    
sentences_val, labels_val = [], []
for sent1, tag_sent1 in zip(sent_val, tag_sent_val):
    tokens, tags = get_token_tag(sent1, tag_sent1, tokenizer)
    sentences_val.append(tokens)
    labels_val.append(tags)
    
sentences_test, labels_test = [], []
for sent1, tag_sent1 in zip(sent_test, tag_sent_test):
    tokens, tags = get_token_tag(sent1, tag_sent1, tokenizer)
    sentences_test.append(tokens)
    labels_test.append(tags)    

In [27]:
sent1 = sent_test[-1]
tag_sent1 = tag_sent_test[-1]
tokens, tags = get_token_tag(sent1, tag_sent1, tokenizer)

In [28]:
tag_sent1

"<카카오뱅크:ORG>는 오는 <13일:DAT>부터 다음 달 <2일:DAT>까지 '가져오기' 기능을 실행하고 이벤트에 참여한 고객에게 <아이패드:POH>, <애플워치:POH>, <편의점 상품권:POH> 등을 제공하는 이벤트도 진행한다.\n"

In [30]:
ii = -1
print(sentences_test[ii])
print(labels_test[ii])
print(tag_sent_test[ii])

['▁카카오', '뱅', '크', '는', '▁오는', '▁13', '일부터', '▁다음', '▁달', '▁2', '일까지', "▁'", '▁가져', '오', '기', "▁'", '▁기능을', '▁실행', '하고', '▁이벤트', '에', '▁참여한', '▁고객에게', '▁아이', '패', '드', '▁', ',', '▁애플', '워', '치', '▁', ',', '▁편의점', '▁상품', '권', '▁등을', '▁제공하는', '▁이벤트', '도', '▁진행한다', '▁', '.']
['B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'B-DAT', 'I-DAT', 'O', 'O', 'B-DAT', 'I-DAT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-POH', 'I-POH', 'I-POH', 'O', 'O', 'B-POH', 'I-POH', 'I-POH', 'O', 'O', 'B-POH', 'I-POH', 'I-POH', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
<카카오뱅크:ORG>는 오는 <13일:DAT>부터 다음 달 <2일:DAT>까지 '가져오기' 기능을 실행하고 이벤트에 참여한 고객에게 <아이패드:POH>, <애플워치:POH>, <편의점 상품권:POH> 등을 제공하는 이벤트도 진행한다.



In [31]:
with open('data/kobert_data_train.pk', 'wb') as f1:
    pk.dump((sentences_train, labels_train), f1)
with open('data/kobert_data_val.pk', 'wb') as f1:
    pk.dump((sentences_val, labels_val), f1)    
with open('data/kobert_data_test.pk', 'wb') as f1:
    pk.dump((sentences_test, labels_test), f1)        

In [32]:
B_sentences_train, B_labels_train = [], []
for sent1, tag_sent1 in zip(sent_train, tag_sent_train):
    tokens, tags = get_token_tag(sent1, tag_sent1, B_tokenizer)
    B_sentences_train.append(tokens)
    B_labels_train.append(tags)
    
B_sentences_val, B_labels_val = [], []
for sent1, tag_sent1 in zip(sent_val, tag_sent_val):
    tokens, tags = get_token_tag(sent1, tag_sent1, B_tokenizer)
    B_sentences_val.append(tokens)
    B_labels_val.append(tags)
    
B_sentences_test, B_labels_test = [], []
for sent1, tag_sent1 in zip(sent_test, tag_sent_test):
    tokens, tags = get_token_tag(sent1, tag_sent1, B_tokenizer)
    B_sentences_test.append(tokens)
    B_labels_test.append(tags)    

In [33]:
with open('data/bert_data_train.pk', 'wb') as f1:
    pk.dump((B_sentences_train, B_labels_train), f1)
with open('data/bert_data_val.pk', 'wb') as f1:
    pk.dump((B_sentences_val, B_labels_val), f1)
with open('data/bert_data_test.pk', 'wb') as f1:
    pk.dump((B_sentences_test, B_labels_test), f1)    

In [34]:
print(sentences_test[-1])
print(labels_test[-1])

['▁카카오', '뱅', '크', '는', '▁오는', '▁13', '일부터', '▁다음', '▁달', '▁2', '일까지', "▁'", '▁가져', '오', '기', "▁'", '▁기능을', '▁실행', '하고', '▁이벤트', '에', '▁참여한', '▁고객에게', '▁아이', '패', '드', '▁', ',', '▁애플', '워', '치', '▁', ',', '▁편의점', '▁상품', '권', '▁등을', '▁제공하는', '▁이벤트', '도', '▁진행한다', '▁', '.']
['B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'B-DAT', 'I-DAT', 'O', 'O', 'B-DAT', 'I-DAT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-POH', 'I-POH', 'I-POH', 'O', 'O', 'B-POH', 'I-POH', 'I-POH', 'O', 'O', 'B-POH', 'I-POH', 'I-POH', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [35]:
print(sent_test[-1])
print(tag_sent_test[-1])

카카오뱅크는 오는 13일부터 다음 달 2일까지 '가져오기' 기능을 실행하고 이벤트에 참여한 고객에게 아이패드, 애플워치, 편의점 상품권 등을 제공하는 이벤트도 진행한다.

<카카오뱅크:ORG>는 오는 <13일:DAT>부터 다음 달 <2일:DAT>까지 '가져오기' 기능을 실행하고 이벤트에 참여한 고객에게 <아이패드:POH>, <애플워치:POH>, <편의점 상품권:POH> 등을 제공하는 이벤트도 진행한다.

